In [1]:
from numpy.fft import fft, ifft, hfft
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy import signal
import import_ipynb
import Helpers

figsize=(10,5)
oversampling_rate=8

importing Jupyter notebook from Helpers.ipynb


# Start with a dry guitar sound

In [ ]:
fs, data = wavfile.read('samples/guitar.wav')
guitar_sample = np.array(data, dtype=np.float32)
guitar_sample /= np.max(np.abs(guitar_sample) )
plt.figure(figsize=figsize)
plt.plot(guitar_sample)
Audio("samples/guitar.wav", autoplay=False)

In [ ]:
plt.figure(figsize=figsize)
plt.plot(np.abs(fft(guitar_sample)), label="dry guitar")
plt.xlim(100 , guitar_sample.size / 2)
plt.xscale("log")
plt.legend()
plt.show()

# Modelling the distortion as a simple static waveshaper

In [ ]:
def distortion(x, n):
     return 2/np.pi * np.arctan(n*x) # Hard clipping distortion

# The harmonics problem

In [ ]:
sine = np.sin(2 * np.pi * 4123 * np.linspace(0, 2, 2 * fs))
sine_disto = distortion(sine, 80)

wavfile.write('samples/sine.wav', fs, (0x7FFF *sine).astype(np.int16))
Audio("samples/sine.wav", autoplay=False)

In [ ]:
plt.figure(figsize=figsize)
plt.plot(np.abs( fft(sine_disto)), label="distorted sine")
plt.plot(np.abs( fft(sine)), label="sine")
plt.xlim(10 * sine.size/2 / 22050 , sine.size /2 ) 
plt.xscale("log")
plt.legend()
plt.show()



wavfile.write('samples/sine_disto.wav', fs, sine_disto)
Audio("samples/sine_disto.wav", autoplay=False)

In [ ]:
sine_oversample = oversample(sine, oversampling_rate)
sine_disto_oversample = distortion(sine_oversample, 50)
sine_disto_downsample = undersample(sine_disto_oversample, oversampling_rate)

In [ ]:
plt.figure(figsize=figsize)
plt.plot(np.abs( fft(sine_disto_downsample)), label="distorted sine")
plt.plot(np.abs( fft(sine)), label="sine")
plt.xlim(10 * sine.size/2 / 22050 , sine.size /2 ) 
plt.xscale("log")
plt.legend()
plt.show()

wavfile.write('samples/sine_disto_downsample.wav', fs, sine_disto_downsample)
Audio("samples/sine_disto_downsample.wav", autoplay=False)

In [ ]:
plt.figure(figsize=figsize)
plt.plot(sine, label="sine")
plt.plot(sine_disto, label="distorted sine")
plt.plot(sine_disto_downsample, label="distorted sine with oversampling")
plt.xlim(0, 20 )
plt.legend()
plt.show()

# Naive waveshaper on guitar (1-band)

In [ ]:
guitar_oversample = oversample(guitar_sample, oversampling_rate)
guitar_disto_oversample = distortion(guitar_oversample, 50)
guitar_disto_naive = undersample(guitar_disto_oversample, oversampling_rate)

plt.figure(figsize=figsize)
plt.plot(guitar_disto_naive)
wavfile.write('samples/guitar_disto_naive.wav', fs, (0x7FFF * guitar_disto_naive).astype(np.int16))
Audio("samples/guitar_disto_naive.wav", autoplay=False)

# 3-band filter

In [ ]:
guitar_low = butter_pass_filter(guitar_oversample,  200., fs*oversampling_rate, btype="low", order=7).astype(np.float32) 
guitar_band = butter_pass_filter(guitar_oversample,  np.array((200., 1000.)), fs*oversampling_rate, btype="band", order=7).astype(np.float32) 
guitar_high = butter_pass_filter(guitar_oversample,  1000., fs*oversampling_rate, btype="high", order=7).astype(np.float32) 

fft_low = np.abs( fft(guitar_low) )
fft_band = np.abs( fft(guitar_band) )
fft_high = np.abs( fft(guitar_high) )

In [ ]:
plt.figure(figsize=figsize)
plt.plot(fft_low, label="lowpass")
plt.plot(fft_band, label="bandpass")
plt.plot(fft_high, label="highpass")
plt.xlim(100, fft_high.size/2 / oversampling_rate) 
plt.xscale("log")
plt.legend()
plt.show()

# Waveshaping each band

In [ ]:
guitar_disto_3band_oversampled = distortion(guitar_low, 30) + distortion(guitar_band, 20) + distortion(guitar_high, 2)
guitar_disto_3band = undersample(guitar_disto_3band_oversampled, oversampling_rate)

plt.figure(figsize=figsize)
plt.figure(figsize=figsize)
plt.plot(guitar_disto_3band)


wavfile.write('samples/guitar_disto_3band.wav', fs, guitar_disto_3band)
Audio("samples/guitar_disto_3band.wav", autoplay=False)